In [18]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt

### Load Data

In [20]:
data = pd.read_excel('resources/fall2017data.xlsx', delimiter=',')
data = data.fillna(0)
data
clean_data = data

In [21]:
data

,Timestamp,Email Address,Student Name,Grade,Family Group Leader,Period 1 (First Choice),Period 1 (Second Choice),Period 1 (Third Choice),Period 2 (First Choice),Period 2 (Second Choice),...,Period 4 (First Choice),Period 4 (Second Choice),Period 4 (Third Choice),Period 7 (First Choice),Period 7 (Second Choice),Period 7 (Third Choice),Period 8 (First Choice),Period 8 (Second Choice),Period 8 (Third Choice),Notes
0,2017-06-14 14:26:24.431000,rkolb@icsd.k12.ny.us,Aaron jorgensen,11.0,Gina kolb,Evolutionary Biology,Evolutionary Biology,Other,HS English (TBA),Geology,...,Economics,Other,Economics,Other,US History & Lit: The 20th Century,US History & Lit: The 20th Century,US History & Lit: The 20th Century,US History & Lit: The 20th Century,US History & Lit: The 20th Century,Other = resource room
1,2017-06-15 08:21:16.889000,simon.warhaft@icsd.k12.ny.us,Abbi Fuller,9.0,Z,HS PE,Portfolio Photo,Portfolio Photo,Geology,Other,...,Beginning Algebra and Trigonometry,MS/HS PE,Intro to Physics & Chemistry,Latin American Studies,African Studies,Latin American Studies,Latin American Studies,African Studies,Latin American Studies,other option for 2nd period is TAing Inquiry &...
2,2017-06-14 15:39:51.924000,ktsapis@icsd.k12.ny.us,Abel Logan,7.0,Kaile,7th-8th Grade Art,Middle School Social Studies with Dan,7th-8th Grade Art,Spanish B,Spanish B,...,Researchers (8th Grade SS),Researchers (8th Grade SS),MS/HS PE,MS/HS PE,Poet's Society,Spanish B,Computer Literacy,Fiber Tech,Fiber Tech,0
3,0,jpaskin@icsd.k12.ny.us,Abeygail Sullivan-Pomeroy,5.0,?,0,0,0,0,0,...,0,0,0,Spanish A,Poet's Society,6th Grade Art,Fiber Tech,MS Chemistry,MS PE,0
4,2017-06-15 21:56:03.811000,nora.schapira@icsd.k12.ny.us,Abiah,12.0,Nora,Portfolio Photo,HS English (TBA),Other,Asia Studies,Drawing & Painting,...,Economics,Economics,Other,Other,Other,Other,Ecology,Ecology,Other,Period 7: Other: TA Spanish B (OK with me)\nWe...
5,2017-06-14 10:20:54.506000,sbokaer@icsd.k12.ny.us,Adina Wilensky,7.0,Sarah Jane Bokaer,French B,French B,French B,Algebra A,MS Social Studies with Dan,...,MS/HS PE,Algebra A,MS Science (TBA),Poet's Society,MS/HS PE,MS/HS PE,MS Chemistry,MS Chemistry,Computer Literacy,"For Period 3, Dan Torrington has arranged for ..."
6,0,jpaskin@icsd.k12.ny.us,Adrian Dvorak,5.0,?,0,0,0,0,0,...,0,0,0,MS/HS PE,6th Grade Art,Spanish A,MS Chemistry,Fiber Tech,MS PE,0
7,2017-06-16 08:59:54.191000,michael.nardi@icsd.k12.ny.us,alana craib,10.0,michael,Facing History,Evolutionary Biology,Evolutionary Biology,Facing History,Asia Studies,...,Intermediate Algebra and Geometry,Intermediate Algebra and Geometry,Intermediate Algebra and Geometry,Molecular Biology,Celtic Band,US History & Lit: The 20th Century,Choir,Dark Fiction,US History & Lit: The 20th Century,0
8,2017-06-16 11:17:26.857000,bronwen.exter@icsd.k12.ny.us,Alannah McGarry,12.0,Bronwen,Constitutional Law/Government,Pre-Calculus,Constitutional Law/Government,HS English (TBA),HS English (TBA),...,Community Service Class,Economics,Community Service Class,Molecular Biology,US History & Lit: The 20th Century,Anatomy & Physiology,Dark Fiction,US History & Lit: The 20th Century,Dark Fiction,"To graduate this year, Alannah's schedule opti..."
9,2017-06-15 22:39:31.978000,nora.schapira@icsd.k12.ny.us,Alec Simmons,12.0,Nora,Pre-Calculus,Other,Pre-Calculus,Other,HS English (TBA),...,Economics,Economics,Other,Anatomy & Physiology,Molecular Biology,Anatomy & Physiology,Dark Fiction,Probability & Statistics,Dark Fiction,Other: Free Period


In [22]:
data.Grade.value_counts()

7.0     44
8.0     41
10.0    41
12.0    39
9.0     39
5.0     37
11.0    37
6.0      5
0.0      1
Name: Grade, dtype: int64

In [40]:
"Other" in list(data.iloc[1])

True

In [41]:
count = 0
for i in range(data.shape[0]):
    if "Other" in list(data.iloc[i]):
        count += 1
print("'Other' is in", str(count), "entries")        

'Other' is in 112 entries


### Data Processing
#### Eliminate Incomplete Entries: May have to figure out a better solution later

In [23]:
for choice in ['First', 'Second', 'Third']:
    for i in [1, 2, 3, 4, 7, 8]:
        clean_data = clean_data.iloc[np.where(clean_data['Period ' + str(i) + ' (' + choice + ' Choice)'] != 0)]
print(clean_data.shape)
# clean_data

(246, 24)


In [13]:
clean_data

,Timestamp,Email Address,Student Name,Grade,Family Group Leader,Period 1 (First Choice),Period 1 (Second Choice),Period 1 (Third Choice),Period 2 (First Choice),Period 2 (Second Choice),...,Period 4 (First Choice),Period 4 (Second Choice),Period 4 (Third Choice),Period 7 (First Choice),Period 7 (Second Choice),Period 7 (Third Choice),Period 8 (First Choice),Period 8 (Second Choice),Period 8 (Third Choice),Notes
0,2017-06-14 14:26:24.431,rkolb@icsd.k12.ny.us,Aaron jorgensen,11.0,Gina kolb,Evolutionary Biology,Evolutionary Biology,Other,HS English (TBA),Geology,...,Economics,Other,Economics,Other,US History & Lit: The 20th Century,US History & Lit: The 20th Century,US History & Lit: The 20th Century,US History & Lit: The 20th Century,US History & Lit: The 20th Century,Other = resource room
1,2017-06-15 08:21:16.889,simon.warhaft@icsd.k12.ny.us,Abbi Fuller,9.0,Z,HS PE,Portfolio Photo,Portfolio Photo,Geology,Other,...,Beginning Algebra and Trigonometry,MS/HS PE,Intro to Physics & Chemistry,Latin American Studies,African Studies,Latin American Studies,Latin American Studies,African Studies,Latin American Studies,other option for 2nd period is TAing Inquiry &...
2,2017-06-14 15:39:51.924,ktsapis@icsd.k12.ny.us,Abel Logan,7.0,Kaile,7th-8th Grade Art,Middle School Social Studies with Dan,7th-8th Grade Art,Spanish B,Spanish B,...,Researchers (8th Grade SS),Researchers (8th Grade SS),MS/HS PE,MS/HS PE,Poet's Society,Spanish B,Computer Literacy,Fiber Tech,Fiber Tech,0
4,2017-06-15 21:56:03.811,nora.schapira@icsd.k12.ny.us,Abiah,12.0,Nora,Portfolio Photo,HS English (TBA),Other,Asia Studies,Drawing & Painting,...,Economics,Economics,Other,Other,Other,Other,Ecology,Ecology,Other,Period 7: Other: TA Spanish B (OK with me)\nWe...
5,2017-06-14 10:20:54.506,sbokaer@icsd.k12.ny.us,Adina Wilensky,7.0,Sarah Jane Bokaer,French B,French B,French B,Algebra A,MS Social Studies with Dan,...,MS/HS PE,Algebra A,MS Science (TBA),Poet's Society,MS/HS PE,MS/HS PE,MS Chemistry,MS Chemistry,Computer Literacy,"For Period 3, Dan Torrington has arranged for ..."
7,2017-06-16 08:59:54.191,michael.nardi@icsd.k12.ny.us,alana craib,10.0,michael,Facing History,Evolutionary Biology,Evolutionary Biology,Facing History,Asia Studies,...,Intermediate Algebra and Geometry,Intermediate Algebra and Geometry,Intermediate Algebra and Geometry,Molecular Biology,Celtic Band,US History & Lit: The 20th Century,Choir,Dark Fiction,US History & Lit: The 20th Century,0
8,2017-06-16 11:17:26.857,bronwen.exter@icsd.k12.ny.us,Alannah McGarry,12.0,Bronwen,Constitutional Law/Government,Pre-Calculus,Constitutional Law/Government,HS English (TBA),HS English (TBA),...,Community Service Class,Economics,Community Service Class,Molecular Biology,US History & Lit: The 20th Century,Anatomy & Physiology,Dark Fiction,US History & Lit: The 20th Century,Dark Fiction,"To graduate this year, Alannah's schedule opti..."
9,2017-06-15 22:39:31.978,nora.schapira@icsd.k12.ny.us,Alec Simmons,12.0,Nora,Pre-Calculus,Other,Pre-Calculus,Other,HS English (TBA),...,Economics,Economics,Other,Anatomy & Physiology,Molecular Biology,Anatomy & Physiology,Dark Fiction,Probability & Statistics,Dark Fiction,Other: Free Period
10,2017-06-16 16:06:55.453,dcowell@icsd.k12.ny.us,Alejandra Colon,9.0,Debbie Cowell,Evolutionary Biology,Other,Portfolio Photo,HS PE,Geology,...,Beginning Algebra and Trigonometry,Beginning Algebra and Trigonometry,Beginning Algebra and Trigonometry,Latin American Studies,Latin American Studies,African Studies,Latin American Studies,Latin American Studies,African Studies,Other = TA I and T
11,2017-06-15 21:42:25.225,naomi.raimon@icsd.k12.ny.us,Alek Travis,10.0,Naomi/Chris,Facing History,Facing History,Advanced Algebra & Trigonometry,Facing History,Facing History,...,Portfolio Art,Community Service Class,Intro to Physics & Chemistry,Advanced Algebra & Trigonometry,Advanced Algebra & Trigonometry,US History & Lit: The 20th Century,Dark Fiction,HS PE,US History & Lit: The 20th Century,Alek really wants t

In [27]:
clean_data.Grade.value_counts()

7.0     44
8.0     41
10.0    41
12.0    39
9.0     39
11.0    37
6.0      5
Name: Grade, dtype: int64

In [42]:
count = 0
for i in range(clean_data.shape[0]):
    if "Other" in list(clean_data.iloc[i]):
        count += 1
print("'Other' is in", str(count), "entries")   

'Other' is in 112 entries


In [66]:
clean_data.columns[5:]

Index(['Period 1 (First Choice)', 'Period 1 (Second Choice)',
       'Period 1 (Third Choice)', 'Period 2 (First Choice)',
       'Period 2 (Second Choice)', 'Period 2 (Third Choice)',
       'Period 3 (First Choice)', 'Period 3 (Second Choice)',
       'Period 3 (Third Choice)', 'Period 4 (First Choice)',
       'Period 4 (Second Choice)', 'Period 4 (Third Choice)',
       'Period 7 (First Choice)', 'Period 7 (Second Choice)',
       'Period 7 (Third Choice)', 'Period 8 (First Choice)',
       'Period 8 (Second Choice)', 'Period 8 (Third Choice)', 'Notes'],
      dtype='object')

In [68]:
num_other = {}
for col in clean_data.columns[5:]:
    num_other[col] = np.sum(clean_data[col]=="Other")

In [71]:
num_other

{'Period 1 (First Choice)': 31,
 'Period 1 (Second Choice)': 38,
 'Period 1 (Third Choice)': 40,
 'Period 2 (First Choice)': 27,
 'Period 2 (Second Choice)': 35,
 'Period 2 (Third Choice)': 41,
 'Period 3 (First Choice)': 24,
 'Period 3 (Second Choice)': 24,
 'Period 3 (Third Choice)': 35,
 'Period 4 (First Choice)': 23,
 'Period 4 (Second Choice)': 33,
 'Period 4 (Third Choice)': 36,
 'Period 7 (First Choice)': 26,
 'Period 7 (Second Choice)': 27,
 'Period 7 (Third Choice)': 36,
 'Period 8 (First Choice)': 25,
 'Period 8 (Second Choice)': 25,
 'Period 8 (Third Choice)': 30}

In [73]:
s = 0
for i in num_other:
    s += num_other[i]
avg = float(s)/len(num_other)
print("On average, each period/choice has", avg, "number of 'other'")

On average, each period/choice has 30.88888888888889 number of 'other'


#### (TODO) Check that all courses requested are in the correct period

#### Make preferences table

In [30]:
# make sure each entry is unique
len(clean_data['Student Name']) == len(set(clean_data['Student Name']))
# student_ids = np.core.defchararray.add('S', np.char.mod('%d', np.arange(len(clean_data['Student Name']))))

True

In [33]:
def all_classes(clean_data):
    # returns a list of all the class names in a period
    result = []
    for period in [1, 2, 3, 4, 7, 8]:
        classes = clean_data.filter(like='Period ' + str(period))
        classes = pd.get_dummies(classes)
        result.extend(classes.columns)
        for choice in ['First', 'Second', 'Third']:
            s = 'Period ' + str(period) + ' (' + choice + ' Choice)'
            s2 = 'P' + str(period)
            result = [c.replace(s, s2) for c in result]
    return result

def preferences(clean_data):
    full_classes = all_classes(clean_data)
    for choice in ['First', 'Second', 'Third']:
        pref = clean_data.filter(like = choice + ' Choice')
        binary = pd.get_dummies(pref)
        cols = binary.columns
        # replace column names
        num_per = 7
        for i in [1, 2, 3, 4, 7, 8]:
            # add classes for each period that were not chosen
            s = 'Period ' + str(i) + ' (' + choice + ' Choice'
            s2 = 'P' + str(i)
            cols = [c.replace(s, s2) for c in cols]
        empty = list(set(full_classes) - set(cols))
        binary.columns = cols
        for e in empty:
            binary[e] = np.zeros(len(clean_data))
        binary = binary.reindex_axis(sorted(binary.columns), axis=1)
        #binary.to_csv('resources/' + choice + 'ChoiceBinary.csv')

preferences(clean_data)

In [34]:
all_classes(clean_data)

['P1_7th-8th Grade Art',
 'P1_Advanced Algebra & Trigonometry',
 'P1_Constitutional Law/Government',
 'P1_Evolutionary Biology',
 'P1_Facing History',
 'P1_French B',
 'P1_HS English (TBA)',
 'P1_HS PE',
 'P1_MS Health',
 'P1_Middle School Social Studies with Dan',
 'P1_Other',
 'P1_Portfolio Photo',
 'P1_Pre-Calculus',
 'P1_Spanish A',
 'P1_7th-8th Grade Art',
 'P1_Advanced Algebra & Trigonometry',
 'P1_Constitutional Law/Government',
 'P1_Evolutionary Biology',
 'P1_Facing History',
 'P1_French B',
 'P1_HS English (TBA)',
 'P1_HS PE',
 'P1_MS Health',
 'P1_Middle School Social Studies with Dan',
 'P1_Other',
 'P1_Portfolio Photo',
 'P1_Pre-Calculus',
 'P1_Spanish A',
 'P1_7th-8th Grade Art',
 'P1_Advanced Algebra & Trigonometry',
 'P1_Algebra A',
 'P1_Constitutional Law/Government',
 'P1_Evolutionary Biology',
 'P1_Facing History',
 'P1_French B',
 'P1_HS English (TBA)',
 'P1_HS PE',
 'P1_MS Health',
 'P1_Middle School Social Studies with Dan',
 'P1_Other',
 'P1_Portfolio Photo',
 'P

In [7]:
first = pd.read_csv('resources/FirstChoiceBinary.csv', delimiter=',')
second = pd.read_csv('resources/SecondChoiceBinary.csv', delimiter=',')
third = pd.read_csv('resources/ThirdChoiceBinary.csv', delimiter=',')

print len(first.columns)
print len(second.columns)
print len(third.columns)

92
92
92


In [11]:
def course_caps(clean_data, class_size):
    n = 8
    period_courses = []
    for i in range(n):
        classes_in_period = clean_data.filter(like='Period ' + str(i + 1))
        classes_in_period = list(set(classes_in_period.values.flatten()))
        
        if 0 in classes_in_period:  # remove the NaNs we cast earlier
            classes_in_period.remove(0)
        
        courses = np.core.defchararray.add(np.repeat("P" + str(i + 1)+'_', len(classes_in_period)) , np.array(classes_in_period))
        period_courses.extend(courses)

    period_courses = pd.DataFrame(period_courses)

    period_courses['Size'] = np.repeat(class_size, len(period_courses))
    period_courses.to_csv('resources/CourseSize.csv')

course_caps(clean_data, 25)

In [13]:
cs = pd.read_csv('resources/CourseSize.csv', delimiter=',')
print cs.shape

(91, 3)


In [26]:
clean_data['Grade'].to_csv('resources/gr.csv')

0      11.0
1       9.0
2       7.0
4      12.0
5       7.0
7      10.0
8      12.0
9      12.0
10      9.0
11     10.0
12      7.0
13      9.0
14      9.0
15      9.0
16      9.0
17      6.0
18     10.0
19      7.0
20     12.0
21      8.0
22     11.0
23      9.0
24      8.0
25      9.0
26     12.0
27     12.0
28     11.0
29     11.0
30     10.0
31      8.0
       ... 
246     7.0
248    11.0
249     8.0
250     9.0
251    11.0
252     9.0
255     7.0
256    10.0
258    10.0
259     8.0
260     7.0
262     7.0
263    12.0
264     8.0
265    12.0
266    11.0
268     8.0
269    10.0
270    10.0
271    12.0
272    11.0
273     7.0
274    10.0
276     9.0
277    10.0
278     7.0
279     7.0
281    10.0
282    11.0
283     9.0
Name: Grade, Length: 246, dtype: float64